# NoteBook train model

## Import thư viện

In [96]:
import pickle
import numpy as np
import cv2
import os

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Dense, MaxPool2D, Conv2D, BatchNormalization, Dropout, Flatten,GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam


## Preprocess data

Xử lý dữ liệu thô

In [97]:
# Lấy thông tin các tệp con (trùng tên với label) trong tệp data_raw_image
list_label = os.listdir('data_image_raw')

# List lưu ảnh đã mã hóa và label tương ứng
data_img = []
label = []

for item in list_label:
    # Tạo dường dẫn đến từng tệp con (có thể dùng cộng xâu bth) cách bên dưới an toàn hơn
    path_label = os.path.join('data_image_raw', item)
    list_image = os.listdir(path_label) # Trả về tên các tệp ảnh
    
    for image in list_image:        # Tạo đường dẫn đến thư mục ảnh
        path_image = os.path.join(path_label, image)
        
        # Đọc ảnh
        matrix = cv2.imread(path_image)
        
        # Thêm ảnh và label tương ứng vào các list
        data_img.append(matrix)
        label.append(item)

    # In ra màn thông báo
    print(f'Đã xử lý xong ảnh của: {item} với số ảnh: {len(list_image)}')

# Chuyển data và label về np.array vì tensorflow yêu cầu đầu vào là np.array, label cần đưa về dạng 2D
data_img = np.array(data_img) 
cat_label = set(label.copy())
label = np.array(label).reshape(-1, 1) # Có thể dùng expand_dim cũng được

# Hiển thị ra màn console
print(f'Shape của data: {data_img.shape} với các label {cat_label}')

Đã xử lý xong ảnh của: Hang với số ảnh: 500
Đã xử lý xong ảnh của: Hieu với số ảnh: 500
Đã xử lý xong ảnh của: Kien với số ảnh: 500
Đã xử lý xong ảnh của: quan với số ảnh: 482
Đã xử lý xong ảnh của: Thanh với số ảnh: 500
Đã xử lý xong ảnh của: Tuan với số ảnh: 500
Đã xử lý xong ảnh của: Viet Anh với số ảnh: 500
Shape của data: (3482, 128, 128, 3) với các label {'Kien', 'quan', 'Hieu', 'Hang', 'Thanh', 'Tuan', 'Viet Anh'}


Lấy data từ file thư mục chứa 

In [98]:
# Encoder label
encoder = OneHotEncoder(sparse_output= False)
label_processed = encoder.fit_transform(label)
# Chuẩn hóa data đầu vào
data_processed = data_img.astype('float32') / 255

In [99]:
data_processed.shape

(3482, 128, 128, 3)

Lưu categories lại vào file để tiện dùng lại để gán label

In [100]:
with open('model/categories.pkl', 'wb') as f:
    pickle.dump(encoder.categories_, f)

In [101]:
with open('model/categories.pkl', 'rb') as f:
    cat = pickle.load(f)

lb = np.array(cat[0])

num_class = lb.size # Số class dùng để phân biệt

Chia tập train test

In [102]:

xtrain, xtest, ytrain, ytest = train_test_split(data_processed, label_processed, test_size= 0.2)

## Tạo model và train

In [103]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.applications import VGG16, EfficientNetB0, MobileNet
from tensorflow.keras.optimizers import Adam

# Load VGG16 without fully connected layers
vgg = MobileNet(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
vgg.trainable = False
# Đóng băng tất cả các tầng trước, sau đó mở khóa một số tầng cuối để fine-tune
for layer in vgg.layers:
    layer.trainable = False  # Ban đầu đóng băng tất cả các tầng

# Mở khóa một số tầng cuối để fine-tune (ví dụ, 4 tầng cuối)
# for layer in vgg.layers[-1:]:
#     layer.trainable = True

# Xây dựng mô hình mới
model_cnn = Sequential([
    vgg,
    GlobalAveragePooling2D(),

    Dense(128, activation='relu'),  # Bổ sung tầng Dense để tăng khả năng học
    Dropout(0.5),

    Dense(num_class, activation='softmax')  # 5 lớp đầu ra
])

# Biên dịch mô hình
model_cnn.compile(
    optimizer=Adam(learning_rate=0.001), 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

# In tóm tắt mô hình
model_cnn.summary()


Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenet_1.00_128 (Functional) │ (None, 4, 4, 1024)     │     3,228,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_23     │ (None, 1024)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 128)            │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,360,967 (12.82 MB)

 Trainable params: 132,103 (516.03 KB)

 Non-trainable params: 3,228,864 (12.32 MB)

Tiến hành train model

In [104]:
model_cnn.fit(xtrain, ytrain, epochs= 4, validation_data = (xtest, ytest) , batch_size= 32)

Epoch 1/4
88/88 ━━━━━━━━━━━━━━━━━━━━ 18s 184ms/step - accuracy: 0.7551 - loss: 0.7557 - val_accuracy: 0.9986 - val_loss: 0.0110
Epoch 2/4
88/88 ━━━━━━━━━━━━━━━━━━━━ 14s 155ms/step - accuracy: 0.9941 - loss: 0.0282 - val_accuracy: 1.0000 - val_loss: 0.0039
Epoch 3/4
88/88 ━━━━━━━━━━━━━━━━━━━━ 14s 158ms/step - accuracy: 0.9982 - loss: 0.0121 - val_accuracy: 1.0000 - val_loss: 0.0020
Epoch 4/4
88/88 ━━━━━━━━━━━━━━━━━━━━ 14s 158ms/step - accuracy: 0.9974 - loss: 0.0116 - val_accuracy: 1.0000 - val_loss: 0.0011


## Lưu model lại

In [105]:
model_cnn.save('model/model_cnn.h5', include_optimizer = True) 

In [106]:
from tensorflow.keras.models import load_model
model = load_model('model/model_cnn.h5')